In [4]:
import tensorflow as tf

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


TensorFlow provides the tf.nn.conv2d() and tf.nn.bias_add() functions to create your own convolutional layers.

The code uses the `tf.nn.conv2d()` function to compute the convolution with weight as the filter and `[1, 2, 2, 1]` for the strides. TensorFlow uses a stride for each input dimension, `[batch, input_height, input_width, input_channels]`. We are generally always going to set the stride for `batch` and `input_channels` (i.e. the first and fourth element in the ` strides` array) to be `1`.

You'll focus on changing `input_height` and `input_width` while setting `batch` and input_channels to 1. The `input_height` and `input_width` strides are for striding the filter over `input`. This example code uses a stride of 2 with 5x5 filter over `input`.

The `tf.nn.bias_add()` function adds a 1-d bias to the last dimension in a matrix.

In [2]:
# Output depth
k_output = 64

# Image Properties
image_width = 10
image_height = 10
color_channels = 3

# Convolution filter
filter_size_width = 5
filter_size_height = 5

# Input/Image
input = tf.placeholder(
    tf.float32,
    shape=[None, image_height, image_width, color_channels])

# Weight and bias
weight = tf.Variable(tf.truncated_normal(
    [filter_size_height, filter_size_width, color_channels, k_output]))
bias = tf.Variable(tf.zeros(k_output))

# Apply Convolution
conv_layer = tf.nn.conv2d(input, weight, strides=[1, 2, 2, 1], padding='SAME')
# Add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
# Apply activation function
conv_layer = tf.nn.relu(conv_layer)

In [3]:
conv_layer = tf.nn.max_pool(
    conv_layer,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='SAME')

The `tf.nn.max_pool()` function performs max pooling with the `ksize` parameter as the size of the filter and the `strides` parameter as the length of the stride. 2x2 filters with a stride of 2x2 are common in practice.

The `ksize` and `strides` parameters are structured as 4-element lists, with each element corresponding to a dimension of the input tensor (`[batch, height, width, channels]`). For both `ksize` and `strides`, the batch and channel dimensions are typically set to 1.

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [8]:
# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units'

In [9]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [10]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [11]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [12]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [13]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch  1, Batch   1 -Loss: 72590.1406 Validation Accuracy: 0.105469
Epoch  1, Batch   2 -Loss: 56762.3359 Validation Accuracy: 0.105469
Epoch  1, Batch   3 -Loss: 59261.1172 Validation Accuracy: 0.113281
Epoch  1, Batch   4 -Loss: 50536.0977 Validation Accuracy: 0.117188
Epoch  1, Batch   5 -Loss: 36053.4297 Validation Accuracy: 0.140625
Epoch  1, Batch   6 -Loss: 37756.0039 Validation Accuracy: 0.148438
Epoch  1, Batch   7 -Loss: 34733.1250 Validation Accuracy: 0.128906
Epoch  1, Batch   8 -Loss: 37447.6641 Validation Accuracy: 0.128906
Epoch  1, Batch   9 -Loss: 32165.2891 Validation Accuracy: 0.167969
Epoch  1, Batch  10 -Loss: 30820.8379 Validation Accuracy: 0.183594
Epoch  1, Batc

Epoch  1, Batch 112 -Loss:  5423.0796 Validation Accuracy: 0.625000
Epoch  1, Batch 113 -Loss:  4968.4883 Validation Accuracy: 0.640625
Epoch  1, Batch 114 -Loss:  5991.4653 Validation Accuracy: 0.636719
Epoch  1, Batch 115 -Loss:  5107.7236 Validation Accuracy: 0.632812
Epoch  1, Batch 116 -Loss:  5179.0879 Validation Accuracy: 0.636719
Epoch  1, Batch 117 -Loss:  4872.7842 Validation Accuracy: 0.644531
Epoch  1, Batch 118 -Loss:  4577.3120 Validation Accuracy: 0.640625
Epoch  1, Batch 119 -Loss:  5699.3213 Validation Accuracy: 0.640625
Epoch  1, Batch 120 -Loss:  5681.1982 Validation Accuracy: 0.640625
Epoch  1, Batch 121 -Loss:  6095.7354 Validation Accuracy: 0.644531
Epoch  1, Batch 122 -Loss:  3937.8254 Validation Accuracy: 0.644531
Epoch  1, Batch 123 -Loss:  5431.5947 Validation Accuracy: 0.660156
Epoch  1, Batch 124 -Loss:  5605.4160 Validation Accuracy: 0.652344
Epoch  1, Batch 125 -Loss:  5637.7295 Validation Accuracy: 0.644531
Epoch  1, Batch 126 -Loss:  5371.7080 Validation

Epoch  1, Batch 233 -Loss:  2433.6375 Validation Accuracy: 0.726562
Epoch  1, Batch 234 -Loss:  3582.3320 Validation Accuracy: 0.722656
Epoch  1, Batch 235 -Loss:  2392.7402 Validation Accuracy: 0.722656
Epoch  1, Batch 236 -Loss:  3226.1316 Validation Accuracy: 0.734375
Epoch  1, Batch 237 -Loss:  2914.1201 Validation Accuracy: 0.726562
Epoch  1, Batch 238 -Loss:  2950.0874 Validation Accuracy: 0.730469
Epoch  1, Batch 239 -Loss:  3116.4521 Validation Accuracy: 0.730469
Epoch  1, Batch 240 -Loss:  2995.3474 Validation Accuracy: 0.730469
Epoch  1, Batch 241 -Loss:  2119.1562 Validation Accuracy: 0.722656
Epoch  1, Batch 242 -Loss:  2876.7605 Validation Accuracy: 0.722656
Epoch  1, Batch 243 -Loss:  2944.5159 Validation Accuracy: 0.722656
Epoch  1, Batch 244 -Loss:  3431.8247 Validation Accuracy: 0.718750
Epoch  1, Batch 245 -Loss:  2576.3098 Validation Accuracy: 0.730469
Epoch  1, Batch 246 -Loss:  3557.9949 Validation Accuracy: 0.730469
Epoch  1, Batch 247 -Loss:  2956.0435 Validation

Epoch  1, Batch 354 -Loss:  2361.8872 Validation Accuracy: 0.781250
Epoch  1, Batch 355 -Loss:  2696.2532 Validation Accuracy: 0.777344
Epoch  1, Batch 356 -Loss:  1676.6952 Validation Accuracy: 0.781250
Epoch  1, Batch 357 -Loss:  2487.6877 Validation Accuracy: 0.777344
Epoch  1, Batch 358 -Loss:  3724.3179 Validation Accuracy: 0.777344
Epoch  1, Batch 359 -Loss:  1875.6826 Validation Accuracy: 0.781250
Epoch  1, Batch 360 -Loss:  3048.4111 Validation Accuracy: 0.777344
Epoch  1, Batch 361 -Loss:  1590.0607 Validation Accuracy: 0.777344
Epoch  1, Batch 362 -Loss:  1799.4261 Validation Accuracy: 0.781250
Epoch  1, Batch 363 -Loss:  1969.6250 Validation Accuracy: 0.777344
Epoch  1, Batch 364 -Loss:  1575.2219 Validation Accuracy: 0.777344
Epoch  1, Batch 365 -Loss:  1566.9406 Validation Accuracy: 0.777344
Epoch  1, Batch 366 -Loss:  2411.1377 Validation Accuracy: 0.777344
Epoch  1, Batch 367 -Loss:  2679.3701 Validation Accuracy: 0.773438
Epoch  1, Batch 368 -Loss:  2567.1313 Validation

Epoch  2, Batch  46 -Loss:  1286.5062 Validation Accuracy: 0.800781
Epoch  2, Batch  47 -Loss:  1419.5483 Validation Accuracy: 0.796875
Epoch  2, Batch  48 -Loss:  1836.1804 Validation Accuracy: 0.796875
Epoch  2, Batch  49 -Loss:  1415.8706 Validation Accuracy: 0.804688
Epoch  2, Batch  50 -Loss:  1945.5103 Validation Accuracy: 0.800781
Epoch  2, Batch  51 -Loss:  1577.1581 Validation Accuracy: 0.808594
Epoch  2, Batch  52 -Loss:  1831.7424 Validation Accuracy: 0.812500
Epoch  2, Batch  53 -Loss:  1712.7159 Validation Accuracy: 0.812500
Epoch  2, Batch  54 -Loss:  1723.5726 Validation Accuracy: 0.816406
Epoch  2, Batch  55 -Loss:  1433.3490 Validation Accuracy: 0.816406
Epoch  2, Batch  56 -Loss:  1686.9426 Validation Accuracy: 0.816406
Epoch  2, Batch  57 -Loss:  1967.8978 Validation Accuracy: 0.820312
Epoch  2, Batch  58 -Loss:  2536.1187 Validation Accuracy: 0.816406
Epoch  2, Batch  59 -Loss:  1390.0154 Validation Accuracy: 0.824219
Epoch  2, Batch  60 -Loss:  1860.3081 Validation

...

Epoch  4, Batch 398 -Loss:   460.2050 Validation Accuracy: 0.886719
Epoch  4, Batch 399 -Loss:   395.6705 Validation Accuracy: 0.890625
Epoch  4, Batch 400 -Loss:   742.9540 Validation Accuracy: 0.890625
Epoch  4, Batch 401 -Loss:   480.3606 Validation Accuracy: 0.890625
Epoch  4, Batch 402 -Loss:   668.6064 Validation Accuracy: 0.890625
Epoch  4, Batch 403 -Loss:   611.3147 Validation Accuracy: 0.890625
Epoch  4, Batch 404 -Loss:   393.6243 Validation Accuracy: 0.886719
Epoch  4, Batch 405 -Loss:   766.7446 Validation Accuracy: 0.886719
Epoch  4, Batch 406 -Loss:   597.9481 Validation Accuracy: 0.886719
Epoch  4, Batch 407 -Loss:   596.8381 Validation Accuracy: 0.886719
Epoch  4, Batch 408 -Loss:   940.9413 Validation Accuracy: 0.886719
Epoch  4, Batch 409 -Loss:   963.3965 Validation Accuracy: 0.886719
Epoch  4, Batch 410 -Loss:   543.7225 Validation Accuracy: 0.886719
Epoch  4, Batch 411 -Loss:   273.0849 Validation Accuracy: 0.886719
Epoch  4, Batch 412 -Loss:   543.2420 Validation

Epoch  5, Batch  90 -Loss:   562.2408 Validation Accuracy: 0.878906
Epoch  5, Batch  91 -Loss:   995.3652 Validation Accuracy: 0.878906
Epoch  5, Batch  92 -Loss:   463.0318 Validation Accuracy: 0.878906
Epoch  5, Batch  93 -Loss:   646.3253 Validation Accuracy: 0.882812
Epoch  5, Batch  94 -Loss:   879.4662 Validation Accuracy: 0.882812
Epoch  5, Batch  95 -Loss:   516.8052 Validation Accuracy: 0.875000
Epoch  5, Batch  96 -Loss:   449.7740 Validation Accuracy: 0.882812
Epoch  5, Batch  97 -Loss:   621.5651 Validation Accuracy: 0.882812
Epoch  5, Batch  98 -Loss:   698.9196 Validation Accuracy: 0.882812
Epoch  5, Batch  99 -Loss:   336.5009 Validation Accuracy: 0.882812
Epoch  5, Batch 100 -Loss:   872.2948 Validation Accuracy: 0.882812
Epoch  5, Batch 101 -Loss:   793.6439 Validation Accuracy: 0.882812
Epoch  5, Batch 102 -Loss:   893.4970 Validation Accuracy: 0.878906
Epoch  5, Batch 103 -Loss:   561.8542 Validation Accuracy: 0.878906
Epoch  5, Batch 104 -Loss:   714.7979 Validation

Epoch  5, Batch 211 -Loss:   717.5174 Validation Accuracy: 0.878906
Epoch  5, Batch 212 -Loss:   377.2409 Validation Accuracy: 0.878906
Epoch  5, Batch 213 -Loss:   459.0957 Validation Accuracy: 0.878906
Epoch  5, Batch 214 -Loss:   402.4963 Validation Accuracy: 0.878906
Epoch  5, Batch 215 -Loss:   802.3749 Validation Accuracy: 0.878906
Epoch  5, Batch 216 -Loss:   538.8304 Validation Accuracy: 0.875000
Epoch  5, Batch 217 -Loss:   580.6185 Validation Accuracy: 0.875000
Epoch  5, Batch 218 -Loss:   410.4775 Validation Accuracy: 0.875000
Epoch  5, Batch 219 -Loss:   523.3701 Validation Accuracy: 0.878906
Epoch  5, Batch 220 -Loss:   450.7278 Validation Accuracy: 0.878906
Epoch  5, Batch 221 -Loss:   546.9090 Validation Accuracy: 0.882812
Epoch  5, Batch 222 -Loss:   803.3687 Validation Accuracy: 0.882812
Epoch  5, Batch 223 -Loss:   419.3982 Validation Accuracy: 0.882812
Epoch  5, Batch 224 -Loss:   652.5103 Validation Accuracy: 0.882812
Epoch  5, Batch 225 -Loss:   782.5627 Validation

Epoch  5, Batch 332 -Loss:   818.7708 Validation Accuracy: 0.882812
Epoch  5, Batch 333 -Loss:   626.8778 Validation Accuracy: 0.882812
Epoch  5, Batch 334 -Loss:   595.7549 Validation Accuracy: 0.882812
Epoch  5, Batch 335 -Loss:   374.9344 Validation Accuracy: 0.886719
Epoch  5, Batch 336 -Loss:   480.8645 Validation Accuracy: 0.886719
Epoch  5, Batch 337 -Loss:   236.3252 Validation Accuracy: 0.886719
Epoch  5, Batch 338 -Loss:   527.7153 Validation Accuracy: 0.882812
Epoch  5, Batch 339 -Loss:   369.6295 Validation Accuracy: 0.886719
Epoch  5, Batch 340 -Loss:   451.0435 Validation Accuracy: 0.886719
Epoch  5, Batch 341 -Loss:   456.6836 Validation Accuracy: 0.886719
Epoch  5, Batch 342 -Loss:   356.9785 Validation Accuracy: 0.886719
Epoch  5, Batch 343 -Loss:   612.2119 Validation Accuracy: 0.886719
Epoch  5, Batch 344 -Loss:   442.0867 Validation Accuracy: 0.886719
Epoch  5, Batch 345 -Loss:   703.9000 Validation Accuracy: 0.886719
Epoch  5, Batch 346 -Loss:   459.0121 Validation

KeyboardInterrupt: 